# Post-processing of micromagnetic simulations in MuMax³: Nanotrack and Nanodisk



# -> Plotting the trajectory of the skyrmion


By means of ```mumax3-convert -comp z -color "blue,white,red" -png -o*.ovf```, the ovf file is converted into a png file, whereby will be used to plot the trajectory of the center of mass of skyrmion throughout the nanotrack. In this case, I chose the color blue to represent the material and red and white to represent the skyrmion nucleus. Furthermore, it is possible to make a video of the simulation by using ```cv2.videoweiter```

In [ ]:
import cv2
import glob 
from natsort import natsorted


class SimulationVideoCreator():
    def __init__(self, input_folder, output_video, fps=30):
        self.input_folder = input_folder
        self.output_video = output_video
        self.fps = fps

    def sort_image_files(self):
        return natsorted(glob.glob(self.input_folder))
    
    def create_video(self):

        image_files = self.sort_image_files()

        first_frame = cv2.imread(image_files[0])
        height, width, layers = first_frame.shape
        video_writer = cv2.VideoWriter(self.output_video, cv2.VideoWriter_fourcc(*'mp4v'), self.fps, (width, height)) #fourcc code for mp4 format (fourcharacter code)

        for image_file in image_files:
            frame = cv2.imread(image_file)

            if frame is not None:
                video_writer.write(frame)
        video_writer.release()

    class PlotTrajectory(SimulationVideoCreator):

     
if __name__ == "__main__":

    input_folder1 = '/home/jfelipe/TCC/nano_pista_semdemag.out/coloridos/*.png' 
    input_folder2 = '/home/jfelipe/TCC/nano_disco_semdemag.out/coloridos/*.png' 
    output_video1 = 'skyrmion_nanotrack_video.mp4'
    output_video2 = 'skyrmion_disk_video.mp4'

    video_creator1 = SimulationVideoCreator(input_folder1, output_video1, fps=30)
    video_creator1.create_video()

    video_creator2 = SimulationVideoCreator(input_folder2, output_video2, fps=30)
    video_creator2.create_video()